# 批量坐标系转换工具 (Batch CRS Transformer)

本工具用于**批量转换矢量数据文件的坐标参考系统（CRS）**，支持递归扫描目录下的所有矢量文件并转换到目标坐标系。

## 主要功能

1. 支持多种矢量格式：Shapefile (.shp)、GeoJSON (.geojson/.json)、GeoPackage (.gpkg)
2. 递归扫描输入目录，自动查找所有矢量文件
3. 自动处理无坐标系的数据（可指定假定源坐标系）
4. 保持原有目录结构，输出到指定目录
5. 跳过已经是目标坐标系的文件

## 典型应用场景

- 将不同坐标系的 GIS 数据统一转换为 WGS84（EPSG:4326）
- 将 WGS84 数据转换为本地投影坐标系（如 CGCS2000）
- 批量处理历史数据的坐标系更新

> **依赖库**  
> - GeoPandas >= 0.14  
> - PyProj >= 3.0  
> 
> 安装命令：  
> ```bash
> pip install geopandas pyproj
> ```

## 1. 导入必要的库

In [ ]:
# ─── ① imports ─────────────────────────────────────────────
import geopandas as gpd
from pathlib import Path
import shutil, os

## 2. 配置转换参数

**重要参数说明：**

- `input_root`: 输入文件根目录（会递归扫描所有子目录）
- `output_root`: 输出文件根目录（保持原目录结构）
- `target_crs`: 目标坐标系（如 `'EPSG:4326'` 表示 WGS84）
- `assumed_source_crs`: 当文件没有坐标系信息时，假定的源坐标系（如 `'EPSG:4547'`）

In [ ]:
# ─── ② 参数配置 ────────────────────────────────────────────
input_root  = Path(r'D:/gis/to_transform')     # 待转换文件根目录
output_root = Path(r'D:/gis/transformed')      # 输出根目录
output_root.mkdir(parents=True, exist_ok=True)

target_crs = 'EPSG:4326'          # 目标：WGS-84
assumed_source_crs = None         # 如果数据本身没 CRS，请在这里填如 'EPSG:4547' 等

## 3. 核心转换函数

该函数执行以下步骤：
1. 读取矢量文件
2. 检查是否有坐标系（没有则使用假定坐标系）
3. 检查是否已经是目标坐标系（是则跳过）
4. 执行坐标转换
5. 保存到输出目录（保持相对路径结构）

In [ ]:
# ─── ③ 核心函数 ────────────────────────────────────────────
def transform_vector(in_path: Path):
    """Read vector file, re-project to `target_crs`, write to mirror location."""
    try:
        gdf = gpd.read_file(in_path)
    except Exception as e:
        print(f'[ERR] {in_path.name}: {e}')
        return
    
    if gdf.crs is None:
        if not assumed_source_crs:
            print(f'[SKIP] {in_path.name}: no CRS & no assumed_source_crs.')
            return
        gdf = gdf.set_crs(assumed_source_crs)
    
    if str(gdf.crs).upper() == str(target_crs).upper():
        print(f'[=] {in_path.name}: already in {target_crs}')
        return
    
    gdf_out = gdf.to_crs(target_crs)
    rel     = in_path.relative_to(input_root)
    out_path = (output_root / rel).with_suffix('.shp')  # 始终写 Shapefile
    out_path.parent.mkdir(parents=True, exist_ok=True)
    gdf_out.to_file(out_path)
    print(f'[OK] {in_path} → {out_path}')

## 4. 批量处理执行

扫描输入目录，找到所有支持的矢量文件并逐个转换。

In [ ]:
# ─── ④ 批处理执行 ──────────────────────────────────────────
vector_exts = {'.shp', '.geojson', '.json', '.gpkg'}
files = [p for p in input_root.rglob('*') if p.suffix.lower() in vector_exts]

print(f'Total vector files: {len(files)}')
for vec in files:
    transform_vector(vec)